In [1]:
from PIL import Image
import os
import shutil
import numpy as np
# import cv2
import re
import pickle
import random
import time
# from tqdm import tqdm
import pandas as pd 
from glob import glob
from xml.etree.ElementTree import parse
# from pascal_voc_writer import Writer
import matplotlib.pyplot as plt

import os
import numpy as np 
import pandas as pd 
from datetime import datetime
import time
import random
from tqdm.auto import tqdm


#Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.optim import Adam, lr_scheduler

#sklearn
from sklearn.model_selection import StratifiedKFold

#CV
import cv2

################# DETR FUCNTIONS FOR LOSS######################## 
import sys
sys.path.extend(['./detr/'])

from models.matcher import HungarianMatcher
from models.detr import SetCriterion
################################################################

#Albumenatations
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2

#Glob
from glob import glob

import sys
sys.path.extend(['./util/'])
from data_parser import *
from image_prepro import *
# from torchvision import transforms

from visualize import *

In [2]:
df = pd.read_csv('df1.csv')
df = df[['file_name', 'res', 'h', 'w', 'xmin', 'ymin', 'xmax', 'ymax', 'bw', 'bh', 'area',
       'label', 'org_label', 'path']]
df.head()

,file_name,res,h,w,xmin,ymin,xmax,ymax,bw,bh,area,label,org_label,path
0,20200513_104735.png,"(1560, 1632)",1560,1632,1040,715,1154,820,114,105,11970,HSIL,HSIL,/home/beomgon/Object_Detection/Dataset/SS/04/2...
1,20200513_104735.png,"(1560, 1632)",1560,1632,1061,845,1140,923,79,78,6162,HSIL,HSIL,/home/beomgon/Object_Detection/Dataset/SS/04/2...
2,20200513_104735.png,"(1560, 1632)",1560,1632,509,697,584,787,75,90,6750,HSIL,HSIL,/home/beomgon/Object_Detection/Dataset/SS/04/2...
3,20200513_145534.png,"(1560, 1632)",1560,1632,865,798,1011,998,146,200,29200,HSIL,HSIL,/home/beomgon/Object_Detection/Dataset/SS/04/2...
4,20200518_104822.png,"(1560, 1632)",1560,1632,515,494,633,621,118,127,14986,HSIL,HSIL,/home/beomgon/Object_Detection/Dataset/SS/04/2...


In [3]:
# set(df['label'])
df[df['xmin'] >= df['xmax']]
df[df['ymin'] >= df['ymax']]

,file_name,res,h,w,xmin,ymin,xmax,ymax,bw,bh,area,label,org_label,path


In [4]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        
    @property
    def avg(self):
        return (self.sum / self.count) if self.count>0 else 0

In [5]:
def label_to_id(x) :
    x = str(x)
    if x == 'ASCUS' :
        return 0
    elif x == 'Carcinoma' :
        return 1
    elif x == 'HSIL' :
        return 2
    elif x == 'LSIL' :
        return 3
    elif x == 'Normal' :
        return 4
    
df['label_id'] = df.label.apply(lambda x : label_to_id(x))
df = df[['file_name', 'res', 'h', 'w', 'xmin', 'ymin', 'xmax', 'ymax', 'bw', 'bh', 'area',
       'label', 'org_label','label_id', 'path']]
df.head()

,file_name,res,h,w,xmin,ymin,xmax,ymax,bw,bh,area,label,org_label,label_id,path
0,20200513_104735.png,"(1560, 1632)",1560,1632,1040,715,1154,820,114,105,11970,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/04/2...
1,20200513_104735.png,"(1560, 1632)",1560,1632,1061,845,1140,923,79,78,6162,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/04/2...
2,20200513_104735.png,"(1560, 1632)",1560,1632,509,697,584,787,75,90,6750,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/04/2...
3,20200513_145534.png,"(1560, 1632)",1560,1632,865,798,1011,998,146,200,29200,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/04/2...
4,20200518_104822.png,"(1560, 1632)",1560,1632,515,494,633,621,118,127,14986,HSIL,HSIL,2,/home/beomgon/Object_Detection/Dataset/SS/04/2...


In [6]:
df_data = df.groupby('path')
images = list(set(df.path))

def get_data(img_id):
    if img_id not in df_data.groups:
        return dict(image_id=img_id, source='', boxes=list())
    
    data  = df_data.get_group(img_id)
    labels = data.label.values
#     source = data.source.values
#     source = np.unique(data.source.values)
#     assert len(source)==1, 'corrupted data: %s image_id has many sources: %s' %(img_id,source)
#     source=source[0]
    boxes = data[['xmin','ymin','bw','bh']].values
#     boxes = list(data.bbox)
    label_id = data.label_id.values
    return dict(image_id = img_id, labels=labels, label_id=label_id, boxes = boxes)

image_list = [get_data(img_id) for img_id in images]

In [7]:
transforms = A.Compose([
#     A.RandomCrop(width=450, height=450),
#     A.HorizontalFlip(p=1),
    A.CenterCrop(1560,1560, True,1),
    A.RandomSizedBBoxSafeCrop(384, 384),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.RandomRain(p=0.1),

    A.pytorch.ToTensor(),
# ], p=1.0, bbox_params=A.BboxParams(format='pascal_voc', min_area=0, min_visibility=0.5))
], p=1.0, bbox_params=A.BboxParams(format='coco', min_area=0, min_visibility=0.5, label_fields=['labels'])) 

In [8]:
# DIR_TRAIN = '../input/global-wheat-detection/train'
class WheatDataset(Dataset):
    def __init__(self,image_list,transforms=None):
        self.images = image_list
        self.transforms = transforms
        self.img_ids = {x['image_id']:i for i,x in enumerate(image_list)}
        
    def get_indices(self,img_ids):
        return [self.img_ids[x] for x in img_ids]
        
    def __len__(self) -> int:
        return len(self.images)
    
    def __getitem__(self,index):
        record = self.images[index]
        image_id = record['image_id']

        image = cv2.imread(image_id, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
#         image = cv2.flip(image, 1)
        image /= 255.0
        
        # DETR takes in data in coco format 
        boxes = record['boxes'] 
        labels = record['label_id']

        if self.transforms:
            sample = {
                'image': image,
                'bboxes': boxes,
                'labels': labels
            }
            sample = self.transforms(**sample)
            image  = sample['image']
            boxes  = sample['bboxes']
            labels = sample['labels']

        _,h,w = image.shape

        boxes = A.augmentations.bbox_utils.normalize_bboxes(sample['bboxes'],rows=h,cols=w)
#         print('boxes', boxes)
        ## detr uses center_x,center_y,width,height !!
        if len(boxes)>0:
            boxes = np.array(boxes)
            boxes[:,2:] /= 2
            boxes[:,:2] += boxes[:,2:]
        else:
            boxes = np.zeros((0,4))
    
        target = {}
        target['boxes'] = torch.as_tensor(boxes,dtype=torch.float32)
        target['labels'] = torch.as_tensor(labels, dtype=torch.int64)
        target['image_id'] = torch.tensor([index])
        
        return image, target, image_id

In [9]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [10]:
batch_size = 32
train_ds = WheatDataset(image_list,transforms)

train_data_loader = DataLoader(
#     torch.utils.data.Subset(train_ds,train_indexes),
    train_ds,
    batch_size=batch_size,
#     shuffle=True,
    shuffle=False,
    num_workers=1,
    collate_fn=collate_fn
#     collate_fn=None
)
class DETRModel(nn.Module):
    def __init__(self,num_classes,num_queries):
        super(DETRModel,self).__init__()
        self.num_classes = num_classes
        self.num_queries = num_queries
        
        self.model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)
        self.in_features = self.model.class_embed.in_features
        
        self.model.class_embed = nn.Linear(in_features=self.in_features,out_features=self.num_classes+1)
        self.model.num_queries = self.num_queries
        
    def forward(self,images):
        return self.model(images)

In [11]:
# class DETRModel(nn.Module):
#     def __init__(self,num_classes,num_queries):
#         super(DETRModel,self).__init__()
#         self.num_classes = num_classes
#         self.num_queries = num_queries
        
#         self.model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)
#         self.in_features = self.model.class_embed.in_features
        
#         self.model.class_embed = nn.Linear(in_features=self.in_features,out_features=self.num_classes+1)
#         self.model.num_queries = self.num_queries
        
#     def forward(self,images):
#         return self.model(images)

In [12]:
n_folds = 5
seed = 42
null_class_coef = 0.5
# num_classes = 1
num_queries = 100
# BATCH_SIZE = 2
LR = 5e-5
lr_dict = {'backbone':0.1,'transformer':1,'embed':1,'final': 5}
epochs = 10
max_norm = 0
model_name = 'detr_resnet50'

In [13]:
'''
code taken from github repo detr , 'code present in engine.py'
'''

# matcher = HungarianMatcher(cost_giou=2,cost_class=1,cost_bbox=5)
matcher = HungarianMatcher()

weight_dict = {'loss_ce': 1, 'loss_bbox': 5 , 'loss_giou': 2}

losses = ['labels', 'boxes', 'cardinality']

In [14]:
num_classes = 5
criterion = SetCriterion(num_classes, 
                         matcher, weight_dict, 
                         eos_coef = null_class_coef, 
                         losses=losses)

model = DETRModel(num_classes=5, num_queries=num_queries)
# model.summary()
learning_rate = 0.00005
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Using cache found in /home/beomgon/.cache/torch/hub/facebookresearch_detr_master


In [15]:
scheduler = lr_scheduler.LambdaLR(
    optimizer=optimizer, lr_lambda=lambda epoch: 1 / ((epoch/4) + 1)
)


In [16]:
device = torch.device('cuda')
criterion = criterion.to(device)
model = DETRModel(num_classes=5, num_queries=num_queries)
model = model.to(device)
model.train()
criterion.train()
tk0 = tqdm(train_data_loader, total=len(train_data_loader),leave=False)
log = None
device='cuda'
im = []
path_list = []
for epoch in range(epochs):
    print('********************************************')
    for step, (images, targets, image_ids) in enumerate(tk0):

        im.append(step)
        path_list.append(image_ids)
        images = list(image.to(device) for image in images)
    #     print(images[0])
    #     print(type(images))
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        output = model(images)

        loss_dict = criterion(output, targets)
        if log is None:
            log = {k:AverageMeter() for k in loss_dict}
            log['total_loss'] = AverageMeter()
            log['avg_prec'] = AverageMeter()

        weight_dict = criterion.weight_dict

        total_loss = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)
        if step % 100 == 0:
            print('total loss :', total_loss)

        optimizer.zero_grad()

        total_loss.backward()

    #     if max_norm > 0:
    #         torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)

        optimizer.step()    
        log['total_loss'].update(total_loss.item(),batch_size)
        for k,v in loss_dict.items():
            log[k].update(v.item(),batch_size)

    #     log['avg_prec'].update(challenge_metric(output,targets),batch_size)

        tk0.set_postfix({k:v.avg for k,v in log.items()})    
    scheduler.step()


Using cache found in /home/beomgon/.cache/torch/hub/facebookresearch_detr_master
  0%|          | 1/585 [00:03<36:48,  3.78s/it, loss_ce=2.72, class_error=65.6, loss_bbox=0.611, loss_giou=1.21, cardinality_error=96.1, total_loss=8.19, avg_prec=0]

total loss : tensor(8.1941, device='cuda:0', grad_fn=<AddBackward0>)


 17%|█▋        | 100/585 [05:22<25:39,  3.17s/it, loss_ce=2.68, class_error=65.3, loss_bbox=0.71, loss_giou=1.19, cardinality_error=94.5, total_loss=8.6, avg_prec=0] 

total loss : tensor(9.1209, device='cuda:0', grad_fn=<AddBackward0>)


 34%|███▍      | 200/585 [10:38<19:27,  3.03s/it, loss_ce=2.68, class_error=64.6, loss_bbox=0.706, loss_giou=1.18, cardinality_error=94.7, total_loss=8.57, avg_prec=0]

...

total loss : tensor(8.7272, device='cuda:0', grad_fn=<AddBackward0>)


 85%|████████▌ | 500/585 [26:36<04:38,  3.27s/it, loss_ce=2.68, class_error=64.6, loss_bbox=0.701, loss_giou=1.18, cardinality_error=94.7, total_loss=8.55, avg_prec=0]

total loss : tensor(8.3995, device='cuda:0', grad_fn=<AddBackward0>)


total loss : tensor(8.1499, device='cuda:0', grad_fn=<AddBackward0>)
total loss : tensor(9.2227, device='cuda:0', grad_fn=<AddBackward0>)
total loss : tensor(8.8120, device='cuda:0', grad_fn=<AddBackward0>)
total loss : tensor(8.8710, device='cuda:0', grad_fn=<AddBackward0>)
total loss : tensor(8.8143, device='cuda:0', grad_fn=<AddBackward0>)
total loss : tensor(8.3150, device='cuda:0', grad_fn=<AddBackward0>)
total loss : tensor(8.0714, device='cuda:0', grad_fn=<AddBackward0>)
total loss : tensor(9.2002, device='cuda:0', grad_fn=<AddBackward0>)
total loss : tensor(8.7782, device='cuda:0', grad_fn=<AddBackward0>)
total loss : tensor(8.1059, device='cuda:0', grad_fn=<AddBackward0>)
total loss : tensor(9.0995, device='cuda:0', grad_fn=<AddBackward0>)
total loss : tensor(9.0575, device='cuda:0', grad_fn=<AddBackward0>)
total loss : tensor(8.2639, device='cuda:0', grad_fn=<AddBackward0>)
total loss : tensor(9.4709, device='cuda:0', grad_fn=<AddBackward0>)
total loss : tensor(9.0383, device

KeyboardInterrupt: 